### Step1-LoadData

In [ ]:
import pickle 
import numpy as np

### read train and test data

In [ ]:
with open("train_qa.txt","rb") as train:
    train_data=pickle.load(train)

In [ ]:
with open("test_qa.txt","rb") as test:
    test_data=pickle.load(test)

### The length will be ?? 

In [ ]:
len(test_data)

In [ ]:
len(train_data)

### what the data look like??

#### The first element in the list is the story the second element is the question of the story the last element is the question answer

In [ ]:
train_data[0]

### Story is

In [ ]:
" ".join(train_data[0][0])

### Question is

In [ ]:
" ".join(train_data[0][1])

### Answer is 

In [ ]:
" ".join(train_data[0][2])

## Now we need to make an vocabulary of our large data to 

In [ ]:
alldata=test_data+train_data

## Will make a set to contains the unique words

In [ ]:
vocab=set()
for story,question,answer in alldata:
    vocab=vocab.union(set(story))
    vocab=vocab.union(set(question))
vocab.add("yes")
vocab.add("no")

# Vocab length is 

In [ ]:
vocableng=len(vocab)+1  ## +1 for keras 

## Get all stories length

In [ ]:
storieslength=[len(i[0]) for i in alldata]

In [ ]:
max_story_len=max(storieslength)

In [ ]:
Qlength=[len(i[1]) for i in alldata]

In [ ]:
max_Qlength_len=max(Qlength)

In [ ]:
max_Qlength_len

In [ ]:
vocab

In [ ]:
## time for preprocessing

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [ ]:
# integer encode sequences of words
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [ ]:
tokenizer.word_index

### Now the time to convert our text train data to numbers using keras tokenizer sequence 

In [ ]:
train_Story_text=[]
train_question_text=[]
for story,question,answer in train_data:
    train_Story_text.append(story)
    train_question_text.append(question)
    

In [ ]:
train_story_Seq=tokenizer.texts_to_sequences(train_Story_text)

In [ ]:
train_story_Seq[0]

In [ ]:
train_Story_text[0]

#### now we need to padding the text to be sure that all the data that can goes to the neural netword are in same shape and length 

In [ ]:
def vectorize_Story(data,word_index=tokenizer.word_index,max_story_len=max_story_len,max_question_len=max_Qlength_len):
    ## Stories
    x=[]
    ## question
    xq=[]
    ## correct answer ()
    y=[]
    
    for story,q,ans in data:
        X=[word_index[word.lower()] for word in story] ## index of each word in story
        XQ=[word_index[word.lower()] for word in q] ## index of each word in question
        Y=np.zeros(len(word_index)+1) ## array of zeros to can contains the reuslts 0 for no one for yes
        Y[word_index[ans]]=1 ## using logic to can assign 1 if result is yes
        ## append the result to  to can genrate the full document
        x.append(X)
        xq.append(XQ)
        y.append(Y)
        
   # RETURN TUPLE FOR UNPACKING
    return (pad_sequences(x, maxlen=max_story_len),pad_sequences(xq, maxlen=max_question_len), np.array(y))
    

In [ ]:
story_train,ques_train,answer_train=vectorize_Story(train_data)

In [ ]:
story_test,ques_test,answer_test=vectorize_Story(test_data)

In [ ]:
story_test

In [ ]:
len(story_test)

In [ ]:
answer_test

In [ ]:
tokenizer.word_index['yes']

In [ ]:
tokenizer.word_index['no']

In [ ]:
sum(answer_test)

# Now the time for build the network
### -input encoder M 
### -input encoder C
### -Question Encoder 
### the network is based on this paper check it :
https://proceedings.neurips.cc/paper/2015/file/8fb21ee7a2207526da55a679f0332de2-Paper.pdf

In [ ]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout
from keras.layers import add, dot, concatenate
from keras.layers import LSTM

In [ ]:
# Place holder for the length of story and questions since our network takes two inputs
input_sequence=Input((max_story_len,))
qu=Input((max_Qlength_len,))


In [ ]:
# size of our vocabs
vocab_size=len(vocab)+1

In [ ]:
#Encoder M
input_encoder_m=Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64)) ## 64 from paper
## add droup out to avoid the over fitting 
input_encoder_m.add(Dropout(0.3))

In [ ]:
#Encoder C
input_encoder_c=Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_Qlength_len))
## add droup out to avoid the over fitting 
input_encoder_c.add(Dropout(0.3))

In [ ]:
question_Encoder=Sequential()
question_Encoder.add(Embedding(input_dim=vocab_size,output_dim=64,input_length=max_Qlength_len))
## add droup out to avoid the over fitting 
question_Encoder.add(Dropout(0.3))

In [ ]:
input_encoded_m=input_encoder_m(input_sequence)
input_encoded_c=input_encoder_c(input_sequence)
question_Encoded=question_Encoder(qu)

In [ ]:
## Now will use the dot product as the paper to can multiplt the m with questions and then passes it to soft max
match=dot([input_encoded_m,question_Encoded],axes=(2,2))
match=Activation('softmax')(match)

In [ ]:
response=add([match,input_encoded_c])
response=Permute((2,1))(response)

In [ ]:
answer=concatenate([response,question_Encoded])

In [ ]:
answer

In [ ]:
answer=LSTM(32)(answer)

In [ ]:
## passes the answer to the softmax to get the results

In [ ]:
answer=Dropout(0.5)(answer)
answer=Dense(vocab_size)(answer)
answer = Activation('softmax')(answer)

In [ ]:
model=Model([input_sequence,qu],answer)

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history=model.fit([story_train,ques_train],answer_train,batch_size=32,epochs=120,validation_data=([story_test,ques_test],answer_test))

In [ ]:
filename = 'chatbot_120_epochs.h5'
model.save(filename)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

### Now will try the model on the test sets but will change the questions and the stories

In [ ]:
test_data[0]

## The question will change here and get them in form of numbers to can predict 
sorry for using the old fuction should build new one for prediction 

In [ ]:
data=(['Mary',
  'got',
  'the',
  'milk',
  'there',
  '.',
  'John',
  'moved',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'John', 'in', 'the', 'bedroom', '?'],
 'no')
storypre,question_pre,res=vectorize_Story([data])

In [ ]:
storypre

In [ ]:
respre=model.predict(([storypre,question_pre]))

In [ ]:
respre.argmax()

In [ ]:
tokenizer.word_index['yes']

In [ ]:
test_data[100]

In [ ]:
data=(['John',
  'took',
  'the',
  'apple',
  'there',
  '.',
  'John',
  'went',
  'to',
  'the',
  'bathroom',
  '.'],
 ['Is', 'John', 'in', 'the', 'bathroom', '?'],
 'no')
storypre,question_pre,res=vectorize_Story([data])

In [ ]:
respre=model.predict(([storypre,question_pre]))
respre.argmax()

In [ ]:
test_data[250]

In [ ]:
data=(['Mary',
  'got',
  'the',
  'apple',
  'there',
  '.',
  'John',
  'moved',
  'to',
  'the',
  'bathroom',
  '.'],
 ['Is', 'John', 'in', 'the', 'office', '?'],
 'no')
storypre,question_pre,res=vectorize_Story([data])

In [ ]:
respre=model.predict(([storypre,question_pre]))
respre.argmax()